In [12]:
import anatomist.api as ana
from soma.qt_gui.qtThread import QtThreadCall
from soma.qt_gui.qt_backend import Qt

a = ana.Anatomist()

from soma import aims
import numpy as np

In [13]:
subject = "197550"
side = "R"
region = "ORBITAL" #"CINGULATE"

path_to_t1mri = f'/neurospin/dico/data/bv_databases/human/not_labeled/hcp/hcp/{subject}/t1mri/BL'

t1mri = a.loadObject(f'{path_to_t1mri}/{subject}.nii.gz') # why t1 ?
t1mri.loadReferentialFromHeader()

if False:
    # view an object in a 4 views block
    block = a.createWindowsBlock(2) # 2 columns
    w1 = a.createWindow("Axial", block=block)
    w2 = a.createWindow("Sagittal", block=block)
    w3 = a.createWindow("Coronal", block=block)
    w4 = a.createWindow("3D", block=block)
    t1mri.addInWindows([w1, w2, w3, w4])
    
    # Load an object
    Rwhite = a.loadObject(f'{path_to_t1mri}/default_analysis/segmentation/mesh/{subject}_Rwhite.gii')
    Lwhite = a.loadObject(f'{path_to_t1mri}/default_analysis/segmentation/mesh/{subject}_Lwhite.gii')
    Rwhite.loadReferentialFromHeader()
    Lwhite.loadReferentialFromHeader()


    w4.addObjects(Rwhite)
    w4.addObjects(Lwhite)

    Rsulci = a.loadObject(f'{path_to_t1mri}/default_analysis/folds/3.1/R{subject}.arg')
    Lsulci = a.loadObject(f'{path_to_t1mri}/default_analysis/folds/3.1/L{subject}.arg')
    Rsulci.loadReferentialFromHeader()
    Lsulci.loadReferentialFromHeader()


    #w3d.addObjects(Rsulci)
    #w3d.addObjects(Lsulci)

    w4.addObjects(Rsulci)
    w4.addObjects(Lsulci)


In [14]:
#for HCP

list_subjects = [499566, 480141, 112314, 199453, 151223, 211114, 592455, 584355, 857263, 213421]
block = a.createWindowsBlock(5) # 5 columns
dic_windows = {}

for subject in list_subjects:
    path_to_t1mri = f'/neurospin/dico/data/bv_databases/human/not_labeled/hcp/hcp/{subject}/t1mri/BL'
    dic_windows[f'w{subject}'] = a.createWindow("3D", block=block)

    dic_windows[f'white_{subject}'] = a.loadObject(f'{path_to_t1mri}/default_analysis/segmentation/mesh/{subject}_{side}white.gii')
    dic_windows[f'white_{subject}'].loadReferentialFromHeader()

    dic_windows[f'sulci_{subject}'] = a.loadObject(f'{path_to_t1mri}/default_analysis/folds/3.1/{side}{subject}.arg')
    dic_windows[f'sulci_{subject}'].loadReferentialFromHeader()

    dic_windows[f'w{subject}'].addObjects([dic_windows[f'white_{subject}'], dic_windows[f'sulci_{subject}']])


memory limit: 41512973107
Reading FGraph version 3.1
bounding box found : 32, 29, 57
                     132, 273, 226
memory limit: 41526365388
Reading FGraph version 3.1
bounding box found : 39, 27, 56
                     140, 277, 221
memory limit: 41541494374
Reading FGraph version 3.1
bounding box found : 27, 31, 52
                     134, 272, 224


: 

In [3]:
from soma import aims

def to_bucket(obj):
    if obj.type() == obj.BUCKET:
        return obj
    avol = a.toAimsObject(obj)
    c = aims.Converter(intype=avol, outtype=aims.BucketMap_VOID)
    abck = c(avol)
    bck = a.toAObject(abck)
    bck.releaseAppRef()
    return bck

id = "197550"
windoforcrop = a.createWindow("3D")
mm_skeleton_path = f"/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/CINGULATE/mask/Rcrops/{id}_cropped_skeleton.nii.gz"

#cropi = to_bucket(a.loadObject(mm_skeleton_path))
#windoforcrop.addObjects(cropi)

In [4]:
def crop_mask(file_src, file_cropped, mask):
    """Crops according to mask"""
    volume = aims.read(file_src)
    print(np.count_nonzero(volume.np))
    if mask:
        mask = aims.read(mask)
        arr = volume.np
        arr_mask = np.asarray(mask)
        arr[arr_mask == 0] = 0
        print(np.count_nonzero(volume.np))
    aims.write(volume, file_cropped)

To prepare te files for the convert_volume_to_bucket for a given brain region

In [6]:
mask_path = f'/neurospin/dico/data/deep_folding/current/datasets/hcp/crops/2mm/{region}/mask/{side}mask_skeleton.nii.gz'
file_src = f'/neurospin/dico/data/deep_folding/current/datasets/hcp/skeletons/2mm/{side}/{side}resampled_skeleton_{subject}.nii.gz'
file_cropped = f'/volatile/ad279118/Figures_report/{subject}_{region}_{side}_cropped_skeleton.nii.gz'
crop_mask(file_src, file_cropped, mask_path)

11893
661


In [7]:
file_src

'/neurospin/dico/data/deep_folding/current/datasets/hcp/skeletons/2mm/L/Lresampled_skeleton_213421.nii.gz'

To do prepare the file to get the buckets for all sulci for on hemisphere for one subject

In [5]:
mask_path = False
file_src = f'/neurospin/dico/data/deep_folding/current/datasets/hcp/skeletons/2mm/{side}/{side}resampled_skeleton_{subject}.nii.gz'
file_cropped = f'/volatile/ad279118/Figures_report/{subject}_{side}_skeleton.nii.gz'
crop_mask(file_src, file_cropped, mask_path)

10560


To do from a terminal with bv bash so as to get the buckets

In [ ]:
"""
cd /volatile/ad279118/deep_folding/deep_folding/brainvisa/utils
python3 convert_volume_to_bucket.py -s '/volatile/ad279118/Figures_report' -t '/volatile/ad279118/Figures_report'
"""

To visualize in the end the region included in the whole brain (sulci point of view)

In [11]:
w5 = a.createWindow("3D")
path_to_bck = f"/volatile/ad279118/Figures_report/Cropped_files/{subject}_{region}_{side}_cropped_skeleton.bck"


bcks = a.loadObject(path_to_bck)
bcks.loadReferentialFromHeader()

path_to_source = f"/volatile/ad279118/Figures_report/Cropped_files/{subject}_{side}_skeleton.bck"

source = a.loadObject(path_to_source)
source.loadReferentialFromHeader()

w5.addObjects(Rwhite)
w5.addObjects(Rsulci)
w5.addObjects(bcks)
w5.addObjects(source)

Position : -3.61077, -45.9965, 23.1891, 0
no position could be read at 933, 678
Position : 86.1, 98.6035, 83.3076, 0
Position : 86.1, 103.712, 67.3424, 0
Position : 107.073, 119.196, 67.2012, 0
Position : 87.2126, 119.96, 67.2012, 0
no position could be read at 1669, 936
no position could be read at 323, 720
no position could be read at 1505, 503
no position could be read at 1495, 566
no position could be read at 1120, 381
no position could be read at 1439, 639
control RefTransControl not found


QLayout: Attempting to add QLayout "" to QWidget "", which already has a layout


no position could be read at 1445, 573
no position could be read at 1873, 605
no position could be read at 1717, 714
Position : 72.9976, 41.3465, 99.1904, 0
no position could be read at 1203, 659
no position could be read at 1436, 720
no position could be read at 1530, 460
no position could be read at 1361, 667
no position could be read at 1192, 851
no position could be read at 768, 474
no position could be read at 2281, 520
no position could be read at 1814, 682
no position could be read at 1551, 355
no position could be read at 1834, 746
no position could be read at 1941, 566
no position could be read at 2109, 635
no position could be read at 420, 577
no position could be read at 712, 441
no position could be read at 1917, 877
no position could be read at 2024, 628
no position could be read at 1897, 758
no position could be read at 2007, 879
no position could be read at 1773, 906
no position could be read at 1169, 858
no position could be read at 1457, 706


In [8]:
path_to_bck

'/volatile/ad279118/Figures_report/Cropped_files/197550_ORBITAL_R_cropped_skeleton.bck'

In [9]:
path_to_source

'/volatile/ad279118/Figures_report/Cropped_files/197550_R_skeleton.bck'